## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 26th Feb. 2nd


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,NaN,Condo/Co-op,1800 N Lauderdale Ave #1417,North Lauderdale,FL,33068.0,175000.0,2.0,2.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/North-Lauderdale/180...,NaN,NaN,N,Y,26.205733,-80.235445


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-31    35
2025-01-30    23
2025-01-27    21
2025-01-28    16
2025-01-29    15
2025-02-03     2
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
115,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936
79,PAST SALE,2025-01-30,Condo/Co-op,2000 S Ocean Dr Unit 16A,Hallandale Beach,FL,33009.0,2725000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,MARMLS,A11656802,N,Y,25.978732,-80.118774
17,PAST SALE,2025-01-28,Condo/Co-op,4240 Galt Ocean Dr #2104,Fort Lauderdale,FL,33308.0,1775000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10477198,N,Y,26.179471,-80.097189
77,PAST SALE,2025-01-30,Condo/Co-op,1850 S Ocean Dr #3304,Hallandale Beach,FL,33009.0,1750000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11671740,N,Y,25.984216,-80.118868
72,PAST SALE,2025-01-29,Condo/Co-op,516 Hendricks Isle Unit 4A,Fort Lauderdale,FL,33301.0,1675000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Beaches MLS,F10464090,N,Y,26.129061,-80.121687
4,PAST SALE,NaT,Condo/Co-op,2110 N Ocean Blvd Unit 12E,Fort Lauderdale,FL,33305.0,1655000.0,3.0,3.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,NaN,NaN,N,Y,26.155533,-80.100936
89,PAST SALE,2025-01-30,Condo/Co-op,1700 S Ocean Blvd Unit 5A,Lauderdale By The Sea,FL,33062.0,1550000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10449464,N,Y,26.209206,-80.092779
10,PAST SALE,2025-01-31,Condo/Co-op,2845 NE 9th St #1003,Fort Lauderdale,FL,33304.0,1450000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Beaches MLS,F10473156,N,Y,26.136658,-80.107327
11,PAST SALE,2025-01-27,Condo/Co-op,4445 El Mar Dr #401,Lauderdale By The Sea,FL,33308.0,1225000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10477418,N,Y,26.191645,-80.096050
24,PAST SALE,2025-01-30,Condo/Co-op,4240 Galt Ocean Dr #1901,Fort Lauderdale,FL,33308.0,1100000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10470561,N,Y,26.179471,-80.097189


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,...,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None
1,PAST SALE,2025-01-30,Condo/Co-op,2000 S Ocean Dr Unit 16A,Hallandale Beach,FL,33009.0,2725000.0,3.0,3.5,...,-80.118774,https://www.redfin.com/FL/Hallandale-Beach/200...,Devin Kay,Douglas Elliman,None,None,Ryan Mendell,Maxwell E Realty Inc,None,None
2,PAST SALE,2025-01-28,Condo/Co-op,4240 Galt Ocean Dr #2104,Fort Lauderdale,FL,33308.0,1775000.0,3.0,3.5,...,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Madeline Camp,"Whitaker Realty, PA",None,None,Barbara Byers,Fisher McClellan Real Estate,None,None
3,PAST SALE,2025-01-30,Condo/Co-op,1850 S Ocean Dr #3304,Hallandale Beach,FL,33009.0,1750000.0,3.0,3.5,...,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Amy McDaniel,Luxury Real Estate Properties,None,None,Amy McDaniel,Luxury Real Estate Properties,None,None
4,PAST SALE,2025-01-29,Condo/Co-op,516 Hendricks Isle Unit 4A,Fort Lauderdale,FL,33301.0,1675000.0,3.0,4.5,...,-80.121687,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Rachel Buemi,D'Angelo Realty Group LLC,None,None,Reilly Bayer,The Keyes Company,None,None
5,PAST SALE,NaT,Condo/Co-op,2110 N Ocean Blvd Unit 12E,Fort Lauderdale,FL,33305.0,1655000.0,3.0,3.0,...,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Seaside Properties Group Inc,None,None,Nicola Higgins,Seaside Properties Group Inc,None,None
6,PAST SALE,2025-01-30,Condo/Co-op,1700 S Ocean Blvd Unit 5A,Lauderdale By The Sea,FL,33062.0,1550000.0,3.0,3.0,...,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Sabah Rab,Apex International Brokerage,None,None,Mariane Tamaz,Coldwell Banker Realty,None,None
7,PAST SALE,2025-01-31,Condo/Co-op,2845 NE 9th St #1003,Fort Lauderdale,FL,33304.0,1450000.0,3.0,3.5,...,-80.107327,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Philip Vimini,Re/Max 1st Choice,None,None,Joseph Ando,One Sotheby's Int'l Realty,None,None
8,PAST SALE,2025-01-27,Condo/Co-op,4445 El Mar Dr #401,Lauderdale By The Sea,FL,33308.0,1225000.0,3.0,2.0,...,-80.096050,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Patricia Post,"Post and Post, Inc.",None,None,Nayla Coto,Sovereign Real Estate Group,None,None
9,PAST SALE,2025-01-30,Condo/Co-op,4240 Galt Ocean Dr #1901,Fort Lauderdale,FL,33308.0,1100000.0,3.0,2.0,...,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Daniel Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Daniel Post,"Post and Post, Inc.",Debra Post PLLC,"Post and Post, Inc."


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

118


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$424,236


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$288


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$50,059,870


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
89
Input Previous Week Condo Average Sales Price:
295086
Input Previous Week Condo Average PSF:
264
Input Previous Week Condo Sales total (ex: 198_000_000)
26_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,The Palms,4112.0,NaN,2001.0,NaN,973.0,5509.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,The Palms,4112.0,NaN,2001.0,NaN,973.0,5509.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None,1,orange
1,PAST SALE,2025-01-30,Condo/Co-op,2000 S Ocean Dr Unit 16A,Hallandale Beach,FL,33009.0,2725000.0,3.0,3.5,2000 OCEAN CONDO,2913.0,NaN,2022.0,NaN,935.0,4734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,MARMLS,A11656802,N,Y,25.978732,-80.118774,https://www.redfin.com/FL/Hallandale-Beach/200...,Devin Kay,Douglas Elliman,None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,2,blue
2,PAST SALE,2025-01-28,Condo/Co-op,4240 Galt Ocean Dr #2104,Fort Lauderdale,FL,33308.0,1775000.0,3.0,3.5,L'ambiance Beach Condo,2823.0,NaN,2003.0,NaN,629.0,2817.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10477198,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Madeline Camp,"Whitaker Realty, PA",None,None,Barbara Byers,Fisher McClellan Real Estate,None,None,3,blue
3,PAST SALE,2025-01-30,Condo/Co-op,1850 S Ocean Dr #3304,Hallandale Beach,FL,33009.0,1750000.0,3.0,3.5,BEACH CLUB CONDO TOWER ONE,2078.0,NaN,2006.0,NaN,842.0,2035.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11671740,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Amy McDaniel,Luxury Real Estate Properties,None,None,Amy McDaniel,Luxury Real Estate Properties,None,None,4,blue
4,PAST SALE,2025-01-29,Condo/Co-op,516 Hendricks Isle Unit 4A,Fort Lauderdale,FL,33301.0,1675000.0,3.0,4.5,Aria,3246.0,NaN,2006.0,NaN,516.0,2400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Beaches MLS,F10464090,N,Y,26.129061,-80.121687,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Rachel Buemi,D'Angelo Realty Group LLC,None,None,Reilly Bayer,The Keyes Company,None,None,5,blue
5,PAST SALE,NaT,Condo/Co-op,2110 N Ocean Blvd Unit 12E,Fort Lauderdale,FL,33305.0,1655000.0,3.0,3.0,NaN,2341.0,NaN,2001.0,NaN,707.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,NaN,NaN,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Seaside Properties Group Inc,None,None,Nicola Higgins,Seaside Properties Group Inc,None,None,6,blue
6,PAST SALE,2025-01-30,Condo/Co-op,1700 S Ocean Blvd Unit 5A,Lauderdale By The Sea,FL,33062.0,1550000.0,3.0,3.0,Cristelle,3200.0,NaN,1999.0,NaN,484.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10449464,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Sabah Rab,Apex International Brokerage,None,None,Mariane Tamaz,Coldwell Banker Realty,None,None,7,blue
7,PAST SALE,2025-01-31,Condo/Co-op,2845 NE 9th St #1003,Fort Lauderdale,FL,33304.0,1450000.0,3.0,3.5,Le Club,2630.0,NaN,2002.0,NaN,551.0,2581.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Beaches MLS,F10473156,N,Y,26.136658,-80.107327,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Philip Vimini,Re/Max 1st Choice,None,None,Joseph Ando,One Sotheby's Int'l Realty,None,None,8,blue
8,PAST SALE,2025-01-27,Condo/Co-op,4445 El Mar Dr #401,Lauderdale By The Sea,FL,33308.0,1225000.0,3.0,2.0,Villas By The Sea,1685.0,NaN,2008.0,NaN,727.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10477418,

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,The Palms,4112.0,NaN,2001.0,NaN,973.0,5509.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_02032024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'The Palms at 2110 N Ocean Blvd Unit 25A in Fort Lauderdale'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,The Palms,4112.0,NaN,2001.0,NaN,973.0,5509.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None,1,orange,The Palms at 2110 N Ocean Blvd Unit 25A in For...
1,PAST SALE,2025-01-30,Condo/Co-op,2000 S Ocean Dr Unit 16A,Hallandale Beach,FL,33009.0,2725000.0,3.0,3.5,2000 OCEAN CONDO,2913.0,NaN,2022.0,NaN,935.0,4734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,MARMLS,A11656802,N,Y,25.978732,-80.118774,https://www.redfin.com/FL/Hallandale-Beach/200...,Devin Kay,Douglas Elliman,None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,2,blue,2000 OCEAN CONDO at 2000 S Ocean Dr Unit 16A i...
2,PAST SALE,2025-01-28,Condo/Co-op,4240 Galt Ocean Dr #2104,Fort Lauderdale,FL,33308.0,1775000.0,3.0,3.5,L'ambiance Beach Condo,2823.0,NaN,2003.0,NaN,629.0,2817.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10477198,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Madeline Camp,"Whitaker Realty, PA",None,None,Barbara Byers,Fisher McClellan Real Estate,None,None,3,blue,L'ambiance Beach Condo at 4240 Galt Ocean Dr #...
3,PAST SALE,2025-01-30,Condo/Co-op,1850 S Ocean Dr #3304,Hallandale Beach,FL,33009.0,1750000.0,3.0,3.5,BEACH CLUB CONDO TOWER ONE,2078.0,NaN,2006.0,NaN,842.0,2035.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11671740,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Amy McDaniel,Luxury Real Estate Properties,None,None,Amy McDaniel,Luxury Real Estate Properties,None,None,4,blue,BEACH CLUB CONDO TOWER ONE at 1850 S Ocean Dr ...
4,PAST SALE,2025-01-29,Condo/Co-op,516 Hendricks Isle Unit 4A,Fort Lauderdale,FL,33301.0,1675000.0,3.0,4.5,Aria,3246.0,NaN,2006.0,NaN,516.0,2400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Beaches MLS,F10464090,N,Y,26.129061,-80.121687,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Rachel Buemi,D'Angelo Realty Group LLC,None,None,Reilly Bayer,The Keyes Company,None,None,5,blue,Aria at 516 Hendricks Isle Unit 4A in Fort Lau...
5,PAST SALE,NaT,Condo/Co-op,2110 N Ocean Blvd Unit 12E,Fort Lauderdale,FL,33305.0,1655000.0,3.0,3.0,NaN,2341.0,NaN,2001.0,NaN,707.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,NaN,NaN,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Seaside Properties Group Inc,None,None,Nicola Higgins,Seaside Properties Group Inc,None,None,6,blue,NaN
6,PAST SALE,2025-01-30,Condo/Co-op,1700 S Ocean Blvd Unit 5A,Lauderdale By The Sea,FL,33062.0,1550000.0,3.0,3.0,Cristelle,3200.0,NaN,1999.0,NaN,484.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10449464,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Sabah Rab,Apex International Brokerage,None,None,Mariane Tamaz,Coldwell Banker Realty,None,None,7,blue,Cristelle at 1700 S Ocean Blvd Unit 5A in Laud...
7,PAST SALE,2025-01-31,Condo/Co-op,2845 NE 9th St #1003,Fort Lauderdale,FL,33304.0,1450000.0,3.0,3.5,Le Club,2630.0,NaN,2002.0,NaN,551.0,2581.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Beaches MLS,F10473156,N,Y,26.136658,-80.107327,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Philip Vimini,Re/Max 1st 

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: The Palms closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $1,100,000 to $4,000,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Lauderdale By The Sea, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 118 condo sales totaling $50,059,870 million from Jan. 26th to Feb. 2nd. The previous week, brokers closed 89 condo sales totaling $26,300,000.

Last week’s units sold for an average of $424,236, higher than the $295,086 average price from the previous week. The average price per square fo

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,The Palms,4112.0,NaN,2001.0,NaN,973.0,5509.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None,1,orange,The Palms at 2110 N Ocean Blvd Unit 25A in For...
1,PAST SALE,2025-01-30,Condo/Co-op,2000 S Ocean Dr Unit 16A,Hallandale Beach,FL,33009.0,2725000.0,3.0,3.5,2000 OCEAN CONDO,2913.0,NaN,2022.0,NaN,935.0,4734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,MARMLS,A11656802,N,Y,25.978732,-80.118774,https://www.redfin.com/FL/Hallandale-Beach/200...,Devin Kay,Douglas Elliman,None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,2,blue,2000 OCEAN CONDO at 2000 S Ocean Dr Unit 16A i...
2,PAST SALE,2025-01-28,Condo/Co-op,4240 Galt Ocean Dr #2104,Fort Lauderdale,FL,33308.0,1775000.0,3.0,3.5,L'ambiance Beach Condo,2823.0,NaN,2003.0,NaN,629.0,2817.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10477198,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Madeline Camp,"Whitaker Realty, PA",None,None,Barbara Byers,Fisher McClellan Real Estate,None,None,3,blue,L'ambiance Beach Condo at 4240 Galt Ocean Dr #...
3,PAST SALE,2025-01-30,Condo/Co-op,1850 S Ocean Dr #3304,Hallandale Beach,FL,33009.0,1750000.0,3.0,3.5,BEACH CLUB CONDO TOWER ONE,2078.0,NaN,2006.0,NaN,842.0,2035.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11671740,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Amy McDaniel,Luxury Real Estate Properties,None,None,Amy McDaniel,Luxury Real Estate Properties,None,None,4,blue,BEACH CLUB CONDO TOWER ONE at 1850 S Ocean Dr ...
4,PAST SALE,2025-01-29,Condo/Co-op,516 Hendricks Isle Unit 4A,Fort Lauderdale,FL,33301.0,1675000.0,3.0,4.5,Aria,3246.0,NaN,2006.0,NaN,516.0,2400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Beaches MLS,F10464090,N,Y,26.129061,-80.121687,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Rachel Buemi,D'Angelo Realty Group LLC,None,None,Reilly Bayer,The Keyes Company,None,None,5,blue,Aria at 516 Hendricks Isle Unit 4A in Fort Lau...
5,PAST SALE,NaT,Condo/Co-op,2110 N Ocean Blvd Unit 12E,Fort Lauderdale,FL,33305.0,1655000.0,3.0,3.0,NaN,2341.0,NaN,2001.0,NaN,707.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,NaN,NaN,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Seaside Properties Group Inc,None,None,Nicola Higgins,Seaside Properties Group Inc,None,None,6,blue,NaN
6,PAST SALE,2025-01-30,Condo/Co-op,1700 S Ocean Blvd Unit 5A,Lauderdale By The Sea,FL,33062.0,1550000.0,3.0,3.0,Cristelle,3200.0,NaN,1999.0,NaN,484.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10449464,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Sabah Rab,Apex International Brokerage,None,None,Mariane Tamaz,Coldwell Banker Realty,None,None,7,blue,Cristelle at 1700 S Ocean Blvd Unit 5A in Laud...
7,PAST SALE,2025-01-31,Condo/Co-op,2845 NE 9th St #1003,Fort Lauderdale,FL,33304.0,1450000.0,3.0,3.5,Le Club,2630.0,NaN,2002.0,NaN,551.0,2581.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Beaches MLS,F10473156,N,Y,26.136658,-80.107327,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Philip Vimini,Re/Max 1st 

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: The Palms closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $1,100,000 to $4,000,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Lauderdale By The Sea, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 118 condo sales totaling $50,059,870 million from Jan. 26th to Feb. 2nd. The previous week, brokers closed 89 condo sales totaling $26,300,000.

Last week’s units sold for an average of $424,236, higher than the $295,086 average price from the previous week. The average price per square fo

In [51]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/2110-N-Ocean-Blvd-33305/unit-25A/home/41998300


In [71]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Hallandale-Beach/2000-S-Ocean-Dr-33009/unit-16A/home/177949707


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/4240-Galt-Ocean-Dr-33308/unit-1901/home/42011282


In [54]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Lauderdale-By-The-Sea/4445-El-Mar-Dr-33308/unit-401/home/194019728


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/2110-N-Ocean-Blvd-33305/unit-25A/home/41998300


In [58]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/1700-S-Ocean-Blvd-33062/unit-5A/home/41975328


In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-30,Condo/Co-op,2110 N Ocean Blvd Unit 25A,Fort Lauderdale,FL,33305.0,4000000.0,4.0,5.5,The Palms,4112.0,NaN,2001.0,NaN,973.0,5509.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Beaches MLS,F10421518,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Douglas Elliman,None,None,Juan Acevedo,One Sotheby's International Realty,None,None,1,orange,The Palms at 2110 N Ocean Blvd Unit 25A in For...
1,PAST SALE,2025-01-30,Condo/Co-op,2000 S Ocean Dr Unit 16A,Hallandale Beach,FL,33009.0,2725000.0,3.0,3.5,2000 OCEAN CONDO,2913.0,NaN,2022.0,NaN,935.0,4734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,MARMLS,A11656802,N,Y,25.978732,-80.118774,https://www.redfin.com/FL/Hallandale-Beach/200...,Devin Kay,Douglas Elliman,None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,2,blue,2000 OCEAN CONDO at 2000 S Ocean Dr Unit 16A i...
2,PAST SALE,2025-01-28,Condo/Co-op,4240 Galt Ocean Dr #2104,Fort Lauderdale,FL,33308.0,1775000.0,3.0,3.5,L'ambiance Beach Condo,2823.0,NaN,2003.0,NaN,629.0,2817.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10477198,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Madeline Camp,"Whitaker Realty, PA",None,None,Barbara Byers,Fisher McClellan Real Estate,None,None,3,blue,L'ambiance Beach Condo at 4240 Galt Ocean Dr #...
3,PAST SALE,2025-01-30,Condo/Co-op,1850 S Ocean Dr #3304,Hallandale Beach,FL,33009.0,1750000.0,3.0,3.5,BEACH CLUB CONDO TOWER ONE,2078.0,NaN,2006.0,NaN,842.0,2035.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11671740,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Amy McDaniel,Luxury Real Estate Properties,None,None,Amy McDaniel,Luxury Real Estate Properties,None,None,4,blue,BEACH CLUB CONDO TOWER ONE at 1850 S Ocean Dr ...
4,PAST SALE,2025-01-29,Condo/Co-op,516 Hendricks Isle Unit 4A,Fort Lauderdale,FL,33301.0,1675000.0,3.0,4.5,Aria,3246.0,NaN,2006.0,NaN,516.0,2400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Beaches MLS,F10464090,N,Y,26.129061,-80.121687,https://www.redfin.com/FL/Fort-Lauderdale/516-...,Rachel Buemi,D'Angelo Realty Group LLC,None,None,Reilly Bayer,The Keyes Company,None,None,5,blue,Aria at 516 Hendricks Isle Unit 4A in Fort Lau...
5,PAST SALE,NaT,Condo/Co-op,2110 N Ocean Blvd Unit 12E,Fort Lauderdale,FL,33305.0,1655000.0,3.0,3.0,NaN,2341.0,NaN,2001.0,NaN,707.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2110...,NaN,NaN,N,Y,26.155533,-80.100936,https://www.redfin.com/FL/Fort-Lauderdale/2110...,Nicola Higgins,Seaside Properties Group Inc,None,None,Nicola Higgins,Seaside Properties Group Inc,None,None,6,blue,NaN
6,PAST SALE,2025-01-30,Condo/Co-op,1700 S Ocean Blvd Unit 5A,Lauderdale By The Sea,FL,33062.0,1550000.0,3.0,3.0,Cristelle,3200.0,NaN,1999.0,NaN,484.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10449464,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Sabah Rab,Apex International Brokerage,None,None,Mariane Tamaz,Coldwell Banker Realty,None,None,7,blue,Cristelle at 1700 S Ocean Blvd Unit 5A in Laud...
7,PAST SALE,2025-01-31,Condo/Co-op,2845 NE 9th St #1003,Fort Lauderdale,FL,33304.0,1450000.0,3.0,3.5,Le Club,2630.0,NaN,2002.0,NaN,551.0,2581.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Beaches MLS,F10473156,N,Y,26.136658,-80.107327,https://www.redfin.com/FL/Fort-Lauderdale/2845...,Philip Vimini,Re/Max 1st 

## Time on Market Calculator

In [72]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 11) ## List (Earlier) date
date2 = datetime(2025, 1, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

141


## Clean CSV for Datawrapper Chart

In [73]:
chart_df = df_top_ten

In [74]:
chart_df = chart_df.fillna(" ")

In [75]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [76]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [77]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [78]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [79]:
chart_df['$/SQUARE FEET'].astype(float)

0    973.0
1    935.0
2    629.0
3    842.0
4    516.0
5    707.0
6    484.0
7    551.0
8    727.0
9    599.0
Name: $/SQUARE FEET, dtype: float64

In [80]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [81]:
csv_date_string = today.strftime("%m_%d_%Y")

In [82]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [83]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-30 00:00:00,2110 N Ocean Blvd Unit 25A in Fort Lauderdale,"$4,000,000",4,5.5,"4,112",2001,$973,Beaches MLS,F10421518,Nicola Higgins Douglas Elliman,Juan Acevedo One Sotheby's International Realt...
1,2025-01-30 00:00:00,2000 S Ocean Dr Unit 16A in Hallandale Beach,"$2,725,000",3,3.5,"2,913",2022,$935,MARMLS,A11656802,Devin Kay Douglas Elliman,Ryan Mendell Maxwell E Realty Inc
2,2025-01-28 00:00:00,4240 Galt Ocean Dr #2104 in Fort Lauderdale,"$1,775,000",3,3.5,"2,823",2003,$629,Beaches MLS,F10477198,"Madeline Camp Whitaker Realty, PA",Barbara Byers Fisher McClellan Real Estate
3,2025-01-30 00:00:00,1850 S Ocean Dr #3304 in Hallandale Beach,"$1,750,000",3,3.5,"2,078",2006,$842,MARMLS,A11671740,Amy McDaniel Luxury Real Estate Properties,Amy McDaniel Luxury Real Estate Properties
4,2025-01-29 00:00:00,516 Hendricks Isle Unit 4A in Fort Lauderdale,"$1,675,000",3,4.5,"3,246",2006,$516,Beaches MLS,F10464090,Rachel Buemi D'Angelo Realty Group LLC,Reilly Bayer The Keyes Company
5,,2110 N Ocean Blvd Unit 12E in Fort Lauderdale,"$1,655,000",3,3.0,"2,341",2001,$707,,,Nicola Higgins Seaside Properties Group Inc,Nicola Higgins Seaside Properties Group Inc
6,2025-01-30 00:00:00,1700 S Ocean Blvd Unit 5A in Lauderdale By The...,"$1,550,000",3,3.0,"3,200",1999,$484,Beaches MLS,F10449464,Sabah Rab Apex International Brokerage,Mariane Tamaz Coldwell Banker Realty
7,2025-01-31 00:00:00,2845 NE 9th St #1003 in Fort Lauderdale,"$1,450,000",3,3.5,"2,630",2002,$551,Beaches MLS,F10473156,Philip Vimini Re/Max 1st Choice,Joseph Ando One Sotheby's Int'l Realty
8,2025-01-27 00:00:00,4445 El Mar Dr #401 in Lauderdale By The Sea,"$1,225,000",3,2.0,"1,685",2008,$727,Beaches MLS,F10477418,"Patricia Post Post and Post, Inc.",Nayla Coto Sovereign Real Estate Group
9,2025-01-30 00:00:00,4240 Galt Ocean Dr #1901 in Fort Lauderdale,"$1,100,000",3,2.0,"1,836",2003,$599,Beaches MLS,F10470561,"Daniel Post Post and Post, Inc. Patricia Post ...","Daniel Post Post and Post, Inc. Debra Post PLL..."
